<a href="https://colab.research.google.com/github/pjd-code/microbiome-tools/blob/main/Qiime_Pre_R_Commands_for_basespace_sequence_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparing the Notebook**

We will be using Google Colab notebooks for our microbiome analysis. There are several advantages to this. First, Colab notebooks are entirly cloud based and will run on any operating system this is important since Qiime2 will only work with unix systems and currently does not run on Windows. Second, it allows us to standardize and share our workflows so that other scientist can more easily replicate (or disprove) our results.

In [ ]:
#mount your google drive with the following command
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#clean up the Colab environment
%cd /content/sample_data
!rm *.csv
!rm *.md
!rm *.json
%cd /content

In [ ]:
#use wget and the specific dropbox url plus any wget flags you want to use
!wget https://www.dropbox.com/s/xsmm369v9vf1pa2/v4_gg_13_5_classifier.qza -P /content/sample_data

In [ ]:
#@title ## File path for metadata file.
#@markdown Import your metadata file then copy the path and past it below.

metadata_file_path = "/content/sample_data/milipede_v4_metadata.txt" #@param {type:"string"}

print(metadata_file_path)

###Download the Material
Click the Play button on the Cell beneath this line of text to get started with the setup. Once the cell has finished running you will see a number next where the play button used to be and the output beneeth the cell, this can be text or visualizations depending on what code was run.

In [ ]:
!git clone https://github.com/pjd-code/millipede_test.git

###Setup
Now that we have the files we need, we are going to install Qiime2 on our notebook. The next step can take up to 15 minutes so while it is running please continue reading about what we will be doing today.

In [ ]:
%run /content/millipede_test/setup_qiime2.py

In [ ]:
%cd /bin

### BaseSpace Sequence Hub CLI
You can work with your BaseSpace Sequence Hub data using the command line interface (CLI). The BaseSpace Sequence Hub CLI supports scripting and programmatic access to BaseSpace Sequence Hub for automation, bulk operations, and other routine functions. It can be used independently or in conjunction with BaseMount.

In [ ]:
#download the base space command line app
!wget "https://launch.basespace.illumina.com/CLI/latest/amd64-linux/bs" -O /bin/bs

#this command changes our basespace file (bs) to an executable
!chmod u+x /bin/bs

In [ ]:
#once we have changed the file type we can run the following command to sign in to basespace from colab
#this will allow us to download directly from basespace
!bs auth

In [ ]:
#check to make sure you are logged in
!bs  whoami

In [ ]:
%%capture
!bs project download --id 308529222 --extension=fastq.gz -o /content/sample_data

###Clean up and organization steps.


In [ ]:
%cd /content/sample_data

In [ ]:
#consolidate in one folder
!mkdir samples 
!find . -name "*.gz" -exec mv "{}" samples \;
!rmdir */
!rm *.json

**Casava 1.8 paired-end demultiplexed fastq**

Format description
In Casava 1.8 demultiplexed (paired-end) format, there are two fastq.gz files for each sample in the study, each containing the forward or reverse reads for that sample. The file name includes the sample identifier. The forward and reverse read file names for a single sample might look like L2S357_15_L001_R1_001.fastq.gz and L2S357_15_L001_R2_001.fastq.gz, respectively. The underscore-separated fields in this file name are:

1. the sample identifier,
2. the barcode sequence or a barcode identifier,
3. the lane number,
4. the direction of the read (i.e. R1 or R2), and
5. the set number.

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /content/sample_data/samples \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

In [ ]:
!qiime tools peek demux-paired-end.qza

In [ ]:
!qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

In [ ]:
# This method denoises paired-end sequences, dereplicates them, and filters chimeras.
# p-trunc-len-f : truncates the 3' end of the forward read sequences due to decrease in quality
# p-trunc-len-r : truncates the 3' end of the reverse read sequences due to decrease in quality
# p-trim-left-f : trims the 5' end of the forward read sequences due to decrease in quality
# p-trim-left-r  : trims the 5' end of the reverse read sequences due to decrease in quality

!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trim-left 0 \
  --p-trunc-len 150 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

In [ ]:
%cd /content/sample_data
!qiime feature-table summarize \
  --i-table table-dada2.qza \
  --o-visualization table-dada2.qzv \
  --m-sample-metadata-file {metadata_file_path}

!qiime feature-table tabulate-seqs \
  --i-data rep-seqs-dada2.qza \
  --o-visualization rep-seqs.qzv

In [ ]:
#allows you to cluster at 97% identity (this is optional)
'''
!qiime vsearch cluster-features-open-reference \
  --i-sequences rep-seqs-dada2.qza \
  --i-table table-dada2.qza \
  --i-reference-sequences /content/millipede_test/se-v4-97-ref-seqs.qza \
  --p-perc-identity 0.97 \
  --p-strand plus \
  --p-threads 0 \
  --o-clustered-table table-clust97.qza \
  --o-clustered-sequences seqs-clust97.qza \
  --o-new-reference-sequences RefSeq-vsearch.qza
  '''

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs-dada2.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier /content/sample_data/v4_gg_13_5_classifier.qza \
  --i-reads rep-seqs-dada2.qza \
  --p-reads-per-batch auto \
  --p-n-jobs -1 \
  --o-classification taxonomy.qza

In [ ]:
!qiime taxa collapse \
--i-table table-dada2.qza \
--i-taxonomy taxonomy.qza \
--p-level 7 \
--o-collapsed-table FeatureTable-TaxaCollapse.qza

In [ ]:
!qiime tools export \
  --input-path FeatureTable-TaxaCollapse.qza \
  --output-path feature-table

In [50]:
!biom convert \
-i /content/sample_data/feature-table/feature-table.biom \
-o table.tsv \
--to-tsv

#We will switch to R for the remaining analysis

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-table table-dada2.qza \
  --i-phylogeny rooted-tree.qza \
  --p-sampling-depth 200 \
  --m-metadata-file {metadata_file_path} \
  --output-dir CoreDiversityMetrics

In [ ]:
!qiime taxa barplot \
--i-table table-dada2.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file {metadata_file_path} \
--o-visualization taxa-barplot.qzv

In [ ]:
!qiime diversity alpha-group-significance \
--i-alpha-diversity /content/sample_data/CoreDiversityMetrics/faith_pd_vector.qza \
--m-metadata-file {metadata_file_path} \
--o-visualization faith-alpha-group-significance.qzv

!qiime diversity alpha-group-significance \
--i-alpha-diversity /content/sample_data/CoreDiversityMetrics/evenness_vector.qza \
--m-metadata-file {metadata_file_path} \
--o-visualization evenness-alpha-group-significance.qzv

In [ ]:
!qiime feature-table core-features \
--i-table FeatureTable-TaxaCollapse.qza \
--p-min-fraction .05 \
--output-dir core